# Urban Morphology

[Urban Morphology](https://en.wikipedia.org/wiki/Urban_morphology)

  `the study of the formation of human settlements and the process of their formation and transformation. The study seeks to understand the spatial structure and character of a metropolitan area, city, town or village by examining the patterns of its component parts`
  
A good reference to get started is the work of [Martin Fleischmann](https://onlinelibrary.wiley.com/doi/full/10.1111/gean.12302).  ([see also](https://github.com/martinfleis/evolution-gean))

For this analysis, I'm also taking some inspiration from [this](https://medium.com/analytics-vidhya/measuring-urban-similarities-in-los-angeles-open-data-arcgis-sklearn-71b3bef53ea6) Medium article.

The spatial unit I want to use for comparison is the Business Improvement District (BID).  Building on the previous notebooks I will demonstrate some ideas to combine datasets to create `measures` for BIDs.  Specifically, I'll look at two types of measures:

  1. Land use using the zoning dataset from City of LA.
  2. Street network complexity via OSM and osmnx.
  
My goal is to demonstrate the ideas, not complete the analysis.  Armed with this information you can do a similar analysis for Neighborhood Councils, for example.

Before I start we need to set the env up.  I like to do (most) all my imports upfront.  I do it with a start.py in my profile_default.  This accomplishes the same thing.

**Note:** This can be a bit slow because it initializes osmnx.  

In [ ]:
#imports
%run start.py

## Setup

I'm adding a couple of utility functions for starters.  If you are doing this **for real** you will starting building a supporting library for these types of utilities.  If you have that one of the first things you do is import them.

In [ ]:
def pprint_bid(gdf):
    """
    Think of a pretty print type of function.
    As you see this function is hardcoded for the dataframe with zoning information.
    Ugly but effective for my purpose.
    """
    return gdf.explore(
        column="ZONE_SMRY", 
         tooltip=["ZONE_CMPLT","ZONE_CLASS", "ZONE_SMRY"], 
         popup=True, 
         #tiles="CartoDB positron", 
         cmap=["red", "blue", "green", "yellow", "purple", "orange", "brown"], 
         style_kwds=dict(color="black") 
    )

def side_by_side(overview_gdf, detailed_gdf):
    """
    This is a common idiom that I use.  It will display two maps side by side
    """
    
    left_output = Output(layout={'border': '1px solid black',
                            'width': '50%'})

    right_output = Output(layout={'border': '1px solid black',
                            'width': '50%'})

    with left_output:
        display(overview_gdf.explore())

    with right_output:
        #display(pretty_bid(detailed_gdf))
        display(detailed_gdf)

    display(HBox([left_output, right_output]))

Background context for our analysis is Business Improvement Districts (BIDs).  Not going to spend much time here since we've looked at BIDs in detail in a prior notebook.

In [ ]:
bids_gdf = gpd.read_file('../data/Business Improvement Districts.zip')

In [ ]:
bids_gdf.columns

# Land Use

In [ ]:
IFrame("https://geohub.lacity.org/datasets/lahub::zoning/explore?location=34.018048%2C-118.412136%2C10.23", width=1200, height=800)

From this site, download the Zoning.zip.  This what we'll use for the land use component.

In [ ]:
%%time
zone_gdf = gpd.read_file('../data/Zoning.zip')

In [ ]:
len(zone_gdf)

In [ ]:
zone_gdf.columns

In [ ]:
zone_gdf.head()

Let's look at the summary descrptions for zones.  This fits with the metrics we're trying to build.

**Note:** You can look at a pdf I put in the data directory, Summary of Zoning Regulations, for more details on what this all means.

In [ ]:
zone_gdf.ZONE_SMRY.value_counts()

We should use some of these for our indicators.

I just want the zone polygons that are in the BIDs.  Simple spatial join accomplishes this.

In [ ]:
biz_zone_intersects_gdf = bids_gdf.to_crs('epsg:2229').overlay(zone_gdf.to_crs('epsg:2229'), 
                                                               how='intersection', 
                                                               keep_geom_type=False)

Quick look at what this gdf contains shows it has what we need.  We can subset on prog_name (i.e., the BID name) to get ZONE_SMRY and geometry.

In [ ]:
biz_zone_intersects_gdf.columns

In [ ]:
len(biz_zone_intersects_gdf)

A much smaller number of the zone polygons are in our BIDs (passes the goofy test).

Next I want to start looking at some of the BIDs.  As usual, my mental map of LA is a bit weak so I'll just pick some:

Wilshire, Venic Beach, Chatsworth, and Hollywood.

In [ ]:
biz_zone_intersects_gdf.prog_name.value_counts()

## Wilshire Center

In [ ]:
wilshire_gdf = bids_gdf.query(f"prog_name == 'WILSHIRE CENTER'")

wilshire_zone_gdf = biz_zone_intersects_gdf.query(f"prog_name == 'WILSHIRE CENTER'").reset_index().drop(columns=['index'])

In [ ]:
side_by_side(wilshire_gdf, pprint_bid(wilshire_zone_gdf))

Not real partial to the layout from .explore with the legend and all, but you get the idea.

In [ ]:
wilshire_zone_gdf.ZONE_SMRY.value_counts()

Remember when we discussed the concept of a utility function?  This next section of code is also a candidate!

In [ ]:
wilshire_zone_gdf['area'] = wilshire_zone_gdf.to_crs("epsg:2229").area

In [ ]:
wilshire_commercial_area = wilshire_zone_gdf.query(f"ZONE_SMRY == 'COMMERCIAL'")['area'].sum() 
wilshire_residential_area = wilshire_zone_gdf.query(f"ZONE_SMRY == 'RESIDENTIAL'")['area'].sum()
wilshire_parking_area = wilshire_zone_gdf.query(f"ZONE_SMRY == 'PARKING'")['area'].sum()
wilshire_open_space_area = wilshire_zone_gdf.query(f"ZONE_SMRY == 'OPEN SPACE'")['area'].sum()
wilshire_area = wilshire_gdf.shape_area.iloc[0]#.to_crs("epsg:2229").area.iloc[0]

In [ ]:
print(f"commercial density: {round((wilshire_commercial_area / wilshire_area), 2)}")

print(f"residential density: {round((wilshire_residential_area / wilshire_area), 2)}")

print(f"parking density: {round((wilshire_parking_area / wilshire_area), 2)}")

print(f"open space density: {round((wilshire_open_space_area / wilshire_area), 2)}")

The astute reader will do the math and wonder...  Why don't they add up to (approx) one?

I don't want to investigate any further!

## Venice Beach

In [ ]:
vb_gdf = bids_gdf.query(f"prog_name == 'VENICE BEACH'").reset_index().drop(columns=['index'])

vb_zone_gdf = biz_zone_intersects_gdf.query(f"prog_name == 'VENICE BEACH'").reset_index().drop(columns=['index'])

In [ ]:
side_by_side(vb_gdf, pprint_bid(vb_zone_gdf))

In [ ]:
vb_zone_gdf.ZONE_SMRY.value_counts()

We already see one difference.  This BID has INDUSTRIAL zoned data, whereas Wilshire did not.

I am going to generate all the percentages one more time (remember this is cut-paste but you can build a function).

In [ ]:
vb_zone_gdf.ZONE_SMRY.value_counts()

In [ ]:
vb_zone_gdf['area'] = vb_zone_gdf.area #.to_crs("epsg:2229").area

vb_commercial_area = vb_zone_gdf.query(f"ZONE_SMRY == 'COMMERCIAL'")['area'].sum() 
vb_residential_area = vb_zone_gdf.query(f"ZONE_SMRY == 'RESIDENTIAL'")['area'].sum()
vb_industrial_area = vb_zone_gdf.query(f"ZONE_SMRY == 'INDUSTRIAL'")['area'].sum()
vb_open_space_area = vb_zone_gdf.query(f"ZONE_SMRY == 'OPEN SPACE'")['area'].sum()
vb_public_facility_area = vb_zone_gdf.query(f"ZONE_SMRY == 'PUBLIC FACILITY'")['area'].sum()
vb_area = vb_gdf.shape_area.iloc[0]#to_crs("epsg:2229").area.iloc[0]

print(f"commercial density: {round((vb_commercial_area / vb_area), 2)}")

print(f"residential density: {round((vb_residential_area / vb_area), 2)}")

print(f"industrial density: {round((vb_industrial_area / vb_area), 2)}")

print(f"open space density: {round((vb_open_space_area / vb_area), 2)}")

print(f"public facility density: {round((vb_public_facility_area / vb_area), 2)}")

In [ ]:
(vb_commercial_area / vb_area) + (vb_residential_area / vb_area) + (vb_industrial_area / vb_area) + (vb_open_space_area / vb_area) + (vb_public_facility_area / vb_area)

Once again, ...  If I had (wanted to spend) more time I would try to understand.

Since I'm just looking for `relative` measurements it may be ok.

## Chatsworth

In [ ]:
chatsworth_gdf = bids_gdf.query(f"prog_name == 'CHATSWORTH'")

chatsworth_zone_gdf = biz_zone_intersects_gdf.query(f"prog_name == 'CHATSWORTH'").reset_index().drop(columns=['index'])

side_by_side(chatsworth_gdf, pprint_bid(chatsworth_zone_gdf))

## Hollywood

In [ ]:
hollywood_gdf = bids_gdf.query(f"prog_name == 'HOLLYWOOD ENTERTAINMENT DISTRICT'")

hollywood_zone_gdf = biz_zone_intersects_gdf.query(f"prog_name == 'HOLLYWOOD ENTERTAINMENT DISTRICT'").reset_index().drop(columns=['index'])

side_by_side(hollywood_gdf, pprint_bid(hollywood_zone_gdf))

## Summary Stats

In [ ]:
biz_zone_intersects_gdf.crs

In [ ]:
len(bids_gdf)

In [ ]:
def commercial_measures(row):
    bid_name = row['prog_name']
    bid_area = row.shape_area
    c_area = biz_zone_intersects_gdf.query(f"prog_name == '{bid_name}'").query(f"ZONE_SMRY == 'COMMERCIAL'").area.sum()
    c_density = round((c_area / bid_area), 2)
    
    return pd.Series([c_area, c_density]) #bid_area#, c_percent

In [ ]:
bids_gdf[['commercial_area', 'c_density']] = bids_gdf.apply(lambda row: commercial_measures(row), axis=1)

When you look bids_gdf the righmost column is the density of commercially zoned space.  You can see the range.

In [ ]:
bids_gdf

Remember the four BIDs I've been looking at?

In [ ]:
my_bids = ['WILSHIRE CENTER', 'VENICE BEACH', 'CHATSWORTH', 'HOLLYWOOD ENTERTAINMENT DISTRICT']
bids_gdf.query(f"prog_name in @my_bids")

So, interesting range of commercial zone percentages in these four zones?

Finally we can gen up a choropleth to show the distribution spatially.

In [ ]:
bids_gdf.explore(
        column="c_density", 
         tooltip=["c_density", "prog_name"], 
         popup=True, 
         tiles="CartoDB positron", 
         cmap='YlOrRd', #["red", "blue", "green", "yellow", "purple", "orange", "brown"], 
         style_kwds=dict(color="black") 
    )

So that is all I'll do with the COMMERCIAL zone measurements.

I suspect one may want to look at INDUSTRIAL too?  What if you wanted to apply this same workflow to Neighborhood Councils and the question was livibility?  You could use RESIDENTIAL and OPEN SPACE? ...  

# Street Networks

Next I want to look at some metrics on street neworks in each of the bids.  I will use [osmnx](https://geoffboeing.com/2016/11/osmnx-python-street-networks/#:~:text=OSMnx%20is%20a%20Python%20package,easily%20analyze%20and%20visualize%20them.) and base this on [a paper](https://appliednetsci.springeropen.com/articles/10.1007/s41109-019-0189-1) by Geoff Boeing.

A few comments are in order:

  1.  As the name implies this package uses OSM.  The city of LA publishes a street network too.  If you want to use their data you should check out [another package](https://pysal.org/notebooks/explore/spaghetti/intro.html).  With some work I suspect you can get to similar measures.
  2.  I will not be completing the whole analysis outlined in the paper.  My objective is to show the basics so you can get started.

osmnx produce plots, results, debugging as it executes.  I've figured out how to turn most of it off, but as you'll see I wasn't completely successful!

In [ ]:
ox.config(log_console=False)

Think utility function and import once again.

In [ ]:
#ox.config(log_console=False)
def side_by_side_plots(fig1, fig2):
    """
    This is a common idiom that I use.  It will display two maps side by side
    """
    #fig1 = ox.plot_graph(osmnx_graph)[0];
    #fig2 = ox.plot_orientation(osmnx_graph.to_undirected())[0];
    left_output = Output(layout={'border': '1px solid black',
                            'width': '50%'})

    right_output = Output(layout={'border': '1px solid black',
                            'width': '50%'})

    with left_output:
        display(fig1)

    with right_output:
        #display(pretty_bid(detailed_gdf))
        display(fig2)

    display(HBox([left_output, right_output]))

## Build the street networks

Let's focus on the four BIDs we've been dealing with up to this point.  I will use the gdf to get the geometry.

In [ ]:
my_bids = ['WILSHIRE CENTER', 'VENICE BEACH', 'CHATSWORTH', 'HOLLYWOOD ENTERTAINMENT DISTRICT']
bids_osmnx_gdf = bids_gdf.query(f"prog_name in @my_bids").reset_index().drop(columns=['index'])
bids_osmnx_gdf

I can now use the geometry with osmnx to get the data in that polygon and build the osmnx graph.

**Note:** The osmnx graph is a link-node structure.  It supports many network oriented functions.  It is easy to transform to a geopandas gdf too.

In [ ]:
g_chatsworth = ox.graph_from_polygon(bids_osmnx_gdf.iloc[0]['geometry'], network_type="drive_service")
g_hollywood = ox.graph_from_polygon(bids_osmnx_gdf.iloc[1]['geometry'], network_type="drive_service")
g_wilshire = ox.graph_from_polygon(bids_osmnx_gdf.iloc[2]['geometry'], network_type="drive_service")
g_vb = ox.graph_from_polygon(bids_osmnx_gdf.iloc[3]['geometry'], network_type="drive_service")


As you'll see next I haven't yet figured out how to control the internal "outputs" of the orientation function.

In [ ]:
chatsworth_plot = ox.plot_graph(g_chatsworth, figsize=(12,12), show=False, close=True)[0];
hollywood_plot = ox.plot_graph(g_hollywood, figsize=(12,12), show=False, close=True)[0]
wilshire_plot = ox.plot_graph(g_wilshire, figsize=(12,12), show=False, close=True)[0]
vb_plot = ox.plot_graph(g_vb, show=False, figsize=(12,12), close=True)[0]

ox.add_edge_bearings(g_chatsworth)
chatsworth_orientation_plot = ox.plot_orientation(g_chatsworth.to_undirected())[0];

ox.add_edge_bearings(g_hollywood)
hollywood_orientation_plot = ox.plot_orientation(g_hollywood.to_undirected())[0];

ox.add_edge_bearings(g_wilshire)
wilshire_orientation_plot = ox.plot_orientation(g_wilshire.to_undirected())[0];

ox.add_edge_bearings(g_vb)
vb_orientation_plot = ox.plot_orientation(g_vb.to_undirected())[0];

At this stage we have what's needed for a simple summary of the street network `complexity`.  [read the docs](https://osmnx.readthedocs.io/en/latest/index.html) and you can find many other capabilities of this package!

## Summarize street measures

Finally, let's look at some of the outputs from osmnx.  

For each of the BIDs we'll look at some output from the graph and two plots, side-by-side.  

In [ ]:
def street_summary(osmnx_graph):
    """
    Use the directed graph to generate statistics.
    Uses the stats module from osmnx.
    """
    total_miles = round((ox.stats.street_length_total(osmnx_graph.to_undirected()) * 0.000621371192), 2)
    total_segments = ox.stats.street_segment_count(osmnx_graph.to_undirected())
    total_intersections = ox.stats.intersection_count(osmnx_graph)
    
    print(f"Number of intersections: {total_intersections}")
    print(f"Number of road segments: {total_segments}")
    print(f"Total miles: {total_miles}")

### Chatsworth

Working our way down the list, Chatsworth is number one.

In [ ]:
street_summary(g_chatsworth)

In [ ]:
side_by_side_plots(chatsworth_plot, chatsworth_orientation_plot)

### Hollywood

In [ ]:
street_summary(g_hollywood)

In [ ]:
side_by_side_plots(hollywood_plot, hollywood_orientation_plot)

### Wilshire

In [ ]:
street_summary(g_wilshire)

In [ ]:
side_by_side_plots(wilshire_plot, wilshire_orientation_plot)

### Venice Beach

In [ ]:
street_summary(g_vb)

In [ ]:
side_by_side_plots(vb_plot, vb_orientation_plot)

## Wrapping up network analysis

As you see this is a quick and dirty network analysis using osm and the osmnx package.  It is just scratching the tip of possiblities.

**Note:** I did not compute intersection density, but I hope you could do it now?

**Note:** I did not add any metrics of road network complexity to bids_gdf for upstream analysis.  You should have the tools for this step too!

# Conclusion

I picked two possible measures of spatial structure, land use (commercial land) and road network complexity. For each of the measures I demonstrated how to construct them and how to add them to the dataframe.  If you were building an analytic product there are several important additions to consider:

  1.  Demographic information - Use census or LA city aggregates
  2.  Come back to the businesses dataset and use these spatial metrics to further analyze buisnessess by NAICS
  
I'm sure there are many other options to consider.

Good luck!